<a href="https://colab.research.google.com/github/25i6/semantic-segmentation/blob/main/notebooks/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!git clone https://github.com/sithu31296/semantic-segmentation
%cd semantic-segmentation
%pip install -e .

Cloning into 'semantic-segmentation'...
remote: Enumerating objects: 801, done.
remote: Counting objects: 100% (798/798), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 801 (delta 485), reused 756 (delta 474), pack-reused 3 (from 1)
Receiving objects: 100% (801/801), 54.99 MiB | 28.18 MiB/s, done.
Resolving deltas: 100% (485/485), done.
/content/semantic-segmentation
Obtaining file:///content/semantic-segmentation
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torchvision import io
from torchvision import transforms as T
from PIL import Image

def show_image(image):
    if image.shape[2] != 3: image = image.permute(1, 2, 0)
    image = Image.fromarray(image.numpy())
    return image

## Show Available Pretrained Models

In [ ]:
from semseg import show_models

show_models()

## Load a Pretrained Model

Download a pretrained model's weights from the result table (ADE20K, CityScapes, ...) and put it in `checkpoints/pretrained/model_name/`.

In [ ]:
%pip install -U gdown

In [ ]:
import gdown
from pathlib import Path

ckpt = Path('./checkpoints/pretrained/segformer')
ckpt.mkdir(exist_ok=True, parents=True)

url = 'https://drive.google.com/uc?id=1-OmW3xRD3WAbJTzktPC-VMOF5WMsN8XT'
output = './checkpoints/pretrained/segformer/segformer.b3.ade.pth'

gdown.download(url, output, quiet=False)

In [ ]:
from semseg.models import *

model = eval('SegFormer')(
    backbone='MiT-B3',
    num_classes=150
)

try:
    model.load_state_dict(torch.load('checkpoints/pretrained/segformer/segformer.b3.ade.pth', map_location='cpu'))
except:
    print("Download a pretrained model's weights from the result table.")
model.eval()

print('Loaded Model')

## Simple Image Inference

### Load Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_path = '/content/drive/MyDrive/2025autum/02_Special_Laboratory/02_kosode/junk_box/14-D-201350-100-0003.jpg'#最後がjpgとかじゃなくて日本語だと一個下のブロックでエラー
image = io.read_image(image_path)
print(image.shape)
show_image(image)

### Preprocess

In [ ]:
# resize
image = T.CenterCrop((512, 512))(image)
# scale to [0.0, 1.0]
image = image.float() / 255
# normalize
image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image)
# add batch size
image = image.unsqueeze(0)
image.shape

上のコメントアウトなし版．

In [ ]:
image = T.CenterCrop((512, 512))(image)
image = image.float() / 255
image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image)
image = image.unsqueeze(0)
image.shape

### Model Forward

In [ ]:
with torch.inference_mode():
    seg = model(image)
seg.shape

### Postprocess

In [ ]:
seg = seg.softmax(1).argmax(1).to(int)
#
seg_unique = seg.unique()
seg.unique()

In [ ]:
from semseg.datasets import *

palette = eval('ADE20K').PALETTE

In [ ]:
seg_map = palette[seg].squeeze().to(torch.uint8)
show_image(seg_map)

評価

In [ ]:
import json
#import numpy as np
#import torch
import cv2
#import matplotlib.pyplot as plt
#from semseg.datasets import ADE20K

def load_labelme_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

def compute_metrics(seg, gt):
    pixel_accuracy = (seg == gt).sum() / gt.size
    intersection = np.logical_and(seg == gt, gt > 0).sum()
    union = np.logical_or(seg > 0, gt > 0).sum()
    miou = intersection / (union + 1e-10)
    return pixel_accuracy, miou

def visualize_segmentation(seg, palette):
    seg_map = np.array(palette)[seg.squeeze()]
    return seg_map.astype(np.uint8)

def overlay_labels(image, seg, seg_unique):
    for label in seg_unique:
        mask = (seg == label)
        y, x = np.where(mask)
        if len(x) > 0 and len(y) > 0:
            cx, cy = int(x.mean()), int(y.mean())
            cv2.putText(image, str(label), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    return image

def main(json_file, seg_tensor):
    labelme_data = load_labelme_json(json_file)
    gt = np.array(labelme_data['shapes'])
    seg = seg_tensor.softmax(1).argmax(1).cpu().numpy()
    seg_unique = np.unique(seg)
    palette = ADE20K.PALETTE
    seg_map = visualize_segmentation(seg, palette)
    map_unique = np.unique(seg_map.reshape(-1, seg_map.shape[-1]), axis=0)
    seg_map = overlay_labels(seg_map, seg, seg_unique)
    pixel_accuracy, miou = compute_metrics(seg, gt)
    print(f'Pixel Accuracy: {pixel_accuracy:.4f}, mIoU: {miou:.4f}')
    plt.imshow(seg_map)
    plt.axis('off')
    plt.show()

# 使用例（適切な json_file と seg_tensor を指定する）
main("/content/drive/MyDrive/2025autum/02_Special_Laboratory/02_kosode/junk_box/14-D-201350-100-0057.json", seg)


追加

In [ ]:
import torch
import numpy as np
import cv2
from semseg.datasets import ADE20K  # 例として ADE20K を利用

# -------------------------
# ① セグメンテーション結果の後処理
# -------------------------
# ※ seg はネットワークの出力テンソルとします（例: [B, num_classes, H, W]）
#    ここではバッチサイズ1を前提としています。

# # 各ピクセルのクラスを softmax と argmax で求める
# seg = seg.softmax(1).argmax(1).to(torch.int)  # seg の shape は [1, H, W] になることを想定
# seg_unique = seg.unique()  # 画像内に現れるユニークなクラス番号

# -------------------------
# ② カラーパレットを用いたセグメンテーションマップの生成
# -------------------------
# ADE20K のパレット（色リスト）を取得
palette = ADE20K.PALETTE
# クラス名が定義されている場合（ADE20K.CLASSES など）：
classes = ADE20K.CLASSES if hasattr(ADE20K, 'CLASSES') else None

# seg の各画素に対してパレットの色を適用
# ※ palette[seg] の結果はテンソルで、形状は [1, H, W, 3] となることを想定
seg_map = palette[seg].squeeze().to(torch.uint8)  # shape: [H, W, 3]

# -------------------------
# ③ NumPy 配列に変換して各クラス領域に注釈を描画
# -------------------------
# テンソルを NumPy 配列へ変換（以降 OpenCV の描画処理のため）
seg_np = seg.squeeze().cpu().numpy()      # [H, W]（クラス番号）
seg_map_np = seg_map.cpu().numpy()          # [H, W, 3]（カラー画像）

# 画像のコピーを用意（注釈描画用）
annotated_img = seg_map_np.copy()

# ユニークな各クラス番号について処理
for cls in seg_unique.tolist():
    # 該当クラスのマスク作成
    mask = (seg_np == cls)
    if np.sum(mask) == 0:
        continue

    # マスク上のピクセル座標を取得
    coords = np.argwhere(mask)
    # 領域の左上と右下の座標（バウンディングボックス）を算出
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)
    # バウンディングボックスの中心点を求める
    center_y, center_x = int((y0 + y1) / 2), int((x0 + x1) / 2)

    # クラス番号と、可能ならタグ名（クラス名）をテキストに
    label_text = f"{cls}"
    if classes is not None and cls < len(classes):
        label_text += f": {classes[cls]}"

    # OpenCV の putText 関数でテキストを描画
    # 第4引数はフォント、第5引数はフォントサイズ、(255,255,255) はテキスト色（白）
    cv2.putText(annotated_img, label_text, (center_x, center_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255),
                thickness=1, lineType=cv2.LINE_AA)

# -------------------------
# ④ 結果の表示
# -------------------------
# ここでは show_image 関数を使って画像を表示します（実装は各自）
show_image(torch.from_numpy(annotated_img))


## Show Available Backbones

In [ ]:
from semseg import show_backbones

show_backbones()

## Show Available Heads

In [ ]:
from semseg import show_heads

show_heads()

## Show Available Datasets

In [ ]:
from semseg import show_datasets

show_datasets()

## Construct a Custom Model

### Choose a Backbone

In [ ]:
from semseg.models.backbones import ResNet

backbone = ResNet('18')

In [ ]:
# init random input batch
x = torch.randn(2, 3, 224, 224)

In [ ]:
# get features from the backbone
features = backbone(x)
for out in features:
    print(out.shape)

### Choose a Head

In [ ]:
from semseg.models.heads import UPerHead

head = UPerHead(backbone.channels, 128, num_classes=10)

In [ ]:
seg = head(features)
seg.shape

In [ ]:
from torch.nn import functional as F
# upsample the output
seg = F.interpolate(seg, size=x.shape[-2:], mode='bilinear', align_corners=False)
seg.shape

Check `semseg/models/custom_cnn.py` and `semseg/models/custom_vit.py` for a complete construction for custom model.